In [1]:
import dill
import itertools
import pandas as pd
import time
import torch
from torch import nn
import torch.optim as optim
from fastai.vision.all import URLs
from train_module import training_functions
from models_module import my_models

LOG_PATH = "./data/optimizing_logs/"
GRAPH_PATH = "./data/graphs/"

In [2]:
in_channels = 3
out_channels = 10
device = "cuda:0"

batch_size = 10
epoch_num = 25

In [3]:
train_losses = [nn.CrossEntropyLoss]
learning_rates = [1e-4,  1e-5]
optimizers = [optim.Adam]#, optim.SGD]
linear_layers_shapes = [[300,200], [300,300]]

In [4]:
path = URLs.IMAGENETTE_160
trainloader, valloader = training_functions.prepare_train_and_val_dls(path, batch_size, size=160)

In [5]:
validation_info = []

In [ ]:
for iteration_number, (loss, learning_rate, optimizer, lin_shapes) in enumerate(
    itertools.product(train_losses, learning_rates, optimizers, linear_layers_shapes)
):
    lin_shapes.append(out_channels)
    net = my_models.VGG(in_channels = in_channels, 
                        out_channels = out_channels,
                        conv_blocks_out_size = [64,128,256,512,512],
                        conv_blocks_amounts = [2,2,2,2,2],
                        linear_layers_out_size = lin_shapes.copy())
    net.to(device)
    
    loss = loss()
    optimizer = optimizer(net.parameters(), lr=learning_rate)
    
    iter_amount = len(train_losses)*len(learning_rates)*len(optimizers)*len(linear_layers_shapes)
    
    info_to_show = f"""Config {iteration_number+1}/{iter_amount}:
    Loss: {loss.__class__},\n Learning_rate: {learning_rate},
    Optimizer: {optimizer},\n linear_layers_out_size:{lin_shapes}"""
    print(info_to_show)
        
    start = time.time()
    net, optimization_history = training_functions.train(net, optimizer,
                                                         loss, epoch_num, 
                                                         trainloader, valloader, 
                                                         device, 
                                                         GRAPH_PATH+f"training_config_{iteration_number}_info.jpg")

    validation_info.append([iteration_number+1, 
                            loss, learning_rate, optimizer, lin_shapes,
                            min(optimization_history["val losses"]), 
                            max(optimization_history["val accuracy"]), 
                            min(optimization_history["train losses"]), 
                            max(optimization_history["train accuracy"])])
    print(f"Time: {time.time()-start}")
    with torch.no_grad():
        torch.save({"model_instance" : net,
                    "loss function" : loss,
                    "learning_rate": learning_rate,
                    "optimizer": optimizer,
                   "lin_shapes": lin_shapes},
                   LOG_PATH+f"model_{iteration_number+1}", 
                   pickle_module=dill)
    

Config 1/4:
    Loss: <class 'torch.nn.modules.loss.CrossEntropyLoss'>,
 Learning_rate: 0.0001,
    Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
),
 linear_layers_out_size:[300, 200, 10]
Time: 1136.4309344291687
Config 2/4:
    Loss: <class 'torch.nn.modules.loss.CrossEntropyLoss'>,
 Learning_rate: 0.0001,
    Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
),
 linear_layers_out_size:[300, 300, 10]
Time: 1160.5978634357452
Config 3/4:
    Loss: <class 'torch.nn.modules.loss.CrossEntropyLoss'>,
 Learning_rate: 1e-05,
    Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0
),
 linear_layers_out_size:[300, 200, 10, 10]
Time: 1168.567194700241
Config 4/4:
    Loss: <class 'torch.nn.modules.loss.CrossEntropyLoss'>,
 Learning_rate: 1e-05,
    Optimizer: A

In [ ]:
res = pd.DataFrame(validation_info, 
                   columns=["model_number", "loss", "learning_rate", "optimizer", "lin_shapes",
                            "best_val_loss", "best_val_acc", "best_train_loss", "best_train_acc"])

In [ ]:
res

In [ ]:
res.loc[res["best_val_acc"]==max(res["best_val_acc"])]